# Gauss elimination and LU decomposition

In [1]:
import numpy as np

### Utility

In [2]:
def generate_matrix(n_cols: int, n_rows: int):
    # Uniform używa rozkładu jednostajnego ciągłego
    return np.random.uniform(low=0.1, high=0.01, size=(n_rows, n_cols))

def generate_vector(n: int):
    return np.random.uniform(low=0.1, high=0.01, size=(n,))

In [3]:
generate_matrix(5, 5), generate_vector(5)

(array([[0.02461908, 0.02376166, 0.03750922, 0.08450792, 0.06400768],
        [0.06116688, 0.05988596, 0.03253187, 0.07311733, 0.01685579],
        [0.03155311, 0.01442802, 0.02966505, 0.03741621, 0.08571038],
        [0.03197542, 0.02755534, 0.09268297, 0.08691851, 0.01945102],
        [0.01582654, 0.01613453, 0.02744073, 0.03698507, 0.09684359]]),
 array([0.04915923, 0.01475601, 0.03896281, 0.0343995 , 0.0305812 ]))

Test

### Gauss elimination

In [4]:
def gauss_elimination(A, b):
    n = len(A)
    
    for i in range(n):
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]


    # iterate columns for last to first
    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]
    
    return b


def gauss_elimination_pivoting(A, b):
    n = len(A)
    pivots = []
    
    for i in range(n):
        pivot_row = np.argmax(np.abs(A[i:, i])) + i
        if pivot_row != i:
            pivots.append((i, pivot_row))
            A[[i, pivot_row]] = A[[pivot_row, i]]
            b[[i, pivot_row]] = b[[pivot_row, i]]
        
        divisor = A[i, i]
        A[i] /= divisor
        b[i] /= divisor
        
        for j in range(i + 1, n):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    # iterate columns for last to first
    for i in range(n - 1, -1, -1):
        for j in range(i):
            multiplier = A[j, i]
            A[j] -= multiplier * A[i]
            b[j] -= multiplier * b[i]

    for i, pivot_row in pivots[::-1]: 
        A[[i, pivot_row]] = A[[pivot_row, i]]
        b[[i, pivot_row]] = b[[pivot_row, i]]

    return b

In [48]:
from numpy.linalg import solve

n = 17 + 9

A = generate_matrix(n, n)
b = generate_vector(n)

result_gauss = gauss_elimination(A, b)
result_gauss_pivot = gauss_elimination_pivoting(A, b)
result_numpy = solve(A, b)

print(result_gauss)
print(result_gauss_pivot)
print(result_numpy)

print(f"Are the results close: {np.allclose(result_gauss, result_numpy)}")
print(f"Are the results close: {np.allclose(result_gauss_pivot, result_numpy)}")

[ -3.05636223   5.56799409   3.81215663 -11.67390448  -4.2061286
  13.706072    -1.65327205  -2.15740216  -1.10972595  -5.97715015
  -3.72650011   3.68828494  -2.55259579  -4.01183463  14.18935692
   6.01205993 -10.30144924  -2.61206745 -11.3229585    0.86903294
  -0.60242481   6.67656235   5.31516895  -0.93838313   7.23231232
  -3.55519469]
[ -3.05636223   5.56799409   3.81215663 -11.67390448  -4.2061286
  13.706072    -1.65327205  -2.15740216  -1.10972595  -5.97715015
  -3.72650011   3.68828494  -2.55259579  -4.01183463  14.18935692
   6.01205993 -10.30144924  -2.61206745 -11.3229585    0.86903294
  -0.60242481   6.67656235   5.31516895  -0.93838313   7.23231232
  -3.55519469]
[ -3.05636223   5.56799409   3.81215663 -11.67390448  -4.2061286
  13.706072    -1.65327205  -2.15740216  -1.10972595  -5.97715015
  -3.72650011   3.68828494  -2.55259579  -4.01183463  14.18935692
   6.01205993 -10.30144924  -2.61206745 -11.3229585    0.86903294
  -0.60242481   6.67656235   5.31516895  -0.93838

### LU decomposition

In [122]:
def lu_decomposition(A):
    n = len(A)
    L = np.eye(n)
    U = A.copy()

    for i in range(n):
        # Update U matrix
        # for j in range(i, n):
        #     U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])

        # # Update L matrix
        # for j in range(i + 1, n):
        #     L[j, i] = (A[j, i] - np.dot(L[j, :i], U[:i, i])) / U[i, i]

        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]

    return L, U


def lu_decomposition_pivoting(A):
    n = len(A)
    L = np.zeros((n, n))
    U = A.copy()
    P = np.eye(n)

    for i in range(n):
        pivot_row = np.argmax(np.abs(U[i:, i])) + i
        if pivot_row != i:
            U[[i, pivot_row]] = U[[pivot_row, i]]
            L[[i, pivot_row]] = L[[pivot_row, i]]
            P[[i, pivot_row]] = P[[pivot_row, i]]

        for j in range(i + 1, n):
            L[j, i] = U[j, i] / U[i, i]
            U[j, i:] -= L[j, i] * U[i, i:]

    L = np.eye(n) + L
    # THIS CALCULATES P @ A = L @ U, but we ned A = P @ L @ U to ocmpare with scipy
    # INVERSE OF PERMUTATION MATRIX IS ITS TRANSPOSE
    return P.T, L, U


# Example usage:
A = generate_matrix(3, 3)

L, U = lu_decomposition(A)
print(np.allclose(L @ U, A))

P, L, U = lu_decomposition_pivoting(A)
print(np.allclose(P @ L @ U, A))

True
True


In [124]:
from scipy.linalg import lu

n = 17 + 9
n = 3

A = generate_matrix(n, n)

P, L, U = lu_decomposition_pivoting(A)
sp_P, sp_L, sp_U = lu(A)


print(L)
print(sp_L)
print(f"Are the results close: {np.allclose(L, sp_L)}, {np.allclose(U, sp_U)}")

[[1.         0.         0.        ]
 [0.37595131 1.         0.        ]
 [0.38441969 0.66637271 1.        ]]
[[1.         0.         0.        ]
 [0.37595131 1.         0.        ]
 [0.38441969 0.66637271 1.        ]]
Are the results close: True, True
